# FMP API and the Journery of the Dividends 
Alrighty, so my buddy and I wanted to  see  if it would be possible to get a list  of all  companies  that pay dividends, filter  the companies based upon some criteria like the dividend amount and dividends being paid  out  in the next three months. We started looking for apis that would could tap   into. We found the benzinga api,  but  a free api license was way too limited,  and  we  couldn't really  understand if we were using   the api  correctly. This led us to look around some more, and  we  stumbled across Financial Modeling Prep  (FMP) API, which is free for the  things in  which  we are interested. Although  the  API documentation and the API  itself aren't as well documented or structured,  after a  little trial and error, I  was able to  come up with  some functions that would allow me to pull that data  inwhich we were interested. Here  is how it went. 

In [38]:
# imports
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from datetime import datetime
import  calendar
import requests
import dotenv
import os
import  pandas as pd
# load api key
dotenv.load_dotenv()
fmp_api_key = os.getenv('FMP_API_KEY')

# helper functions for time calculations

def get_todays_date():
    """
    Returns today's date in the format YYYY-DD-MM
    """
    return datetime.now().strftime("%Y-%d-%m")

def get_future_date(start_date_str, months=3):
    """
    Returns a date 'months' months in the future from the 'start_date_str'.
    The 'start_date_str' should be in the format YYYY-DD-MM.
    """
    # Parsing the input date string to a datetime object
    start_date = datetime.strptime(start_date_str, "%Y-%d-%m")

    # Calculating the year and month for the future date
    month = start_date.month - 1 + months
    year = start_date.year + month // 12
    month = month % 12 + 1
    day = min(start_date.day, calendar.monthrange(year, month)[1])

    # Constructing the future date
    future_date = datetime(year, month, day)

    return future_date.strftime("%Y-%d-%m")

In [2]:
#  api functions
def  get_historical_stock_price_for_symbol(symbol:str="AAPL",
                                           headers:dict={"accept": "application/json"},
                                           from_date="",
                                           to_date="",
                                           fmp_api_key = os.getenv('FMP_API_KEY')):
    
    url="https://financialmodelingprep.com/api/v3/historical-price-full/"
    # I don't  think  it  was  working  with the  query  parameter for the request function :(
    if  not from_date or not to_date: 
        url=url+f"{symbol}"+f"?apikey={fmp_api_key}"
    else:
        url=url+f"{symbol}"+f"?from={from_date}"+f"&to_date={to_date}"+f"&apikey={fmp_api_key}"
    
    print(f"the url we are using: {url}")
    response = requests.request("GET", url, headers=headers)

    print(response.text)
    return  response


def get_historical_dividends_for_symbol(symbol:str="AAPL", 
                                        headers:dict={"accept": "application/json"},
                                        fmp_api_key:str=os.getenv('FMP_API_KEY')):

    url="https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/"    
    url=url+f"{symbol}"+f"?apikey={fmp_api_key}"

    print(f"the url we are using: {url}")
    response = requests.request("GET", url, headers=headers)

    print(response.text)
    return  response

def get_company_info(symbol:str="AAPL", 
                     headers:dict={"accept": "application/json"},
                     fmp_api_key:str=os.getenv('FMP_API_KEY')):
    
    url="https://financialmodelingprep.com/api/v3/profile/"
    #TODO: do  some  checking or lookup  on the symbol
    url=url+f"{symbol}"+f"?apikey={fmp_api_key}"
    
    print(f"the url we are using: {url}")
    response = requests.request("GET", url, headers=headers)

    print(response.text)
    return  response

def get_dividends_for_dates(from_date:str, 
                            to_date:str,
                            headers = {"accept": "application/json"},
                            fmp_api_key:str=os.getenv('FMP_API_KEY')):
    
    url = "https://financialmodelingprep.com/api/v3/stock_dividend_calendar?"
    query={ "from":from_date, "to":to_date, "apikey":fmp_api_key}
    
    response = requests.request("GET", url, headers=headers, params=query)
    print(response.text)
    return  response


# First Prototype of Proposed Solution
Here  I  am  going to try and go  through the process that we described for the dividend app.
- 1. Get all dividends for the next 6 months
- 2. filter basedd upon some criteria i.e. stock names
- 3. plot historical dividend prices for the stock

Alrighty, let's see  how this works

In [3]:
# getting dividends for the next  six  months
dividends_next_six_months=get_dividends_for_dates(from_date=get_todays_date(),  to_date=get_future_date(get_todays_date(),  months=6))

[
  {
    "date": "2024-04-30",
    "label": "April 30, 24",
    "adjDividend": 0,
    "symbol": "MMK.VI",
    "dividend": null,
    "recordDate": null,
    "paymentDate": null,
    "declarationDate": null
  },
  {
    "date": "2024-04-30",
    "label": "April 30, 24",
    "adjDividend": 0.5236,
    "symbol": "SDVKF",
    "dividend": 0.5236,
    "recordDate": "2024-05-02",
    "paymentDate": "2024-05-07",
    "declarationDate": null
  },
  {
    "date": "2024-04-30",
    "label": "April 30, 24",
    "adjDividend": 0,
    "symbol": "MNHFF",
    "dividend": null,
    "recordDate": null,
    "paymentDate": null,
    "declarationDate": null
  },
  {
    "date": "2024-04-30",
    "label": "April 30, 24",
    "adjDividend": 0,
    "symbol": "MYM.DE",
    "dividend": null,
    "recordDate": null,
    "paymentDate": null,
    "declarationDate": null
  },
  {
    "date": "2024-04-30",
    "label": "April 30, 24",
    "adjDividend": 5.5,
    "symbol": "SAND.ST",
    "dividend": 5.5,
    "recordD

In [5]:

# filter  for a stock  name
appl=[]
for dividend in dividends_next_six_months.json():
    if dividend["symbol"]=="AAPL":
        print(dividend)
        appl.append(dividend)

Okay, so this resulted  in nothing. It seemst that AAPL is not registered as having any dividends in the near future. I've already come up the issue that filtering on the amount doesn't work because the currency is not always dollars, because the stock exchange could be any of a number of different stock exchanges. 

So, I'll get the in for for all the stocks that  have upcoming dividends so that I can filter for the  stock  exchange and/or the currency. I mean the stock exchange itself is probably bound to a particular currency.  Let's just see what I get.

In [8]:
df=pd.DataFrame(dividends_next_six_months.json())
df.head()

,date,label,adjDividend,symbol,dividend,recordDate,paymentDate,declarationDate
0,2024-04-30,"April 30, 24",0.0000,MMK.VI,NaN,None,None,None
1,2024-04-30,"April 30, 24",0.5236,SDVKF,0.5236,2024-05-02,2024-05-07,None
2,2024-04-30,"April 30, 24",0.0000,MNHFF,NaN,None,None,None
3,2024-04-30,"April 30, 24",0.0000,MYM.DE,NaN,None,None,None
4,2024-04-30,"April 30, 24",5.5000,SAND.ST,5.5000,2024-05-02,2024-05-07,None


In [ ]:
def company_info_for_symbol(symbol:str):
    return get_company_info(symbol=symbol)
infos=[]
for ith,row in df.iterrows():
    infos.append(get_company_info(symbol=row["symbol"]))

Okay, here's  some nice information. The free plan seems to be limited to U.S. stocks only. And to  parse all of  the symbols for all upcoming dividends in the next six months takes some time. At least I know what the currency is and that I can filter for the  dividend price. Also, it seems that using getting stock information on all upcoming stocks has made me reach the  end of my free subscription. 

In [14]:
stock_infos=[]
for stock in infos:

    if "Error Message" in stock.json():
        pass
    else:
        stock_infos.append(stock.json())

In [23]:
stock_info_df=pd.DataFrame().from_dict(stock_infos,)

It seems that the stock information comes back as a dictionary in a list. I need to flatten before I can pop this into a dataframe and merge with the dividen calendar data frame.

In [26]:
flat_stock_infos = [d[0] for  d in  stock_infos]

In [28]:
stock_info_df=pd.DataFrame(flat_stock_infos)

In [ ]:
stock_info_df

In [40]:
combined_df=pd.merge(left=stock_info_df, right=df, on="symbol")

In [47]:
combined_df["currency"].unique()

array(['USD', 'JPY', 'SEK', 'BRL', 'EUR'], dtype=object)

Let's do the filtering. We can create a couple of filters. One filter is for the currency to be "USD" and the dividend is greater  than "1".

In [54]:
currency_is_usd=combined_df["currency"] == "USD"
last_dividend_gt_3=combined_df["dividend"]>1
combined_df[currency_is_usd & last_dividend_gt_3]

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,cik,isin,cusip,exchange,exchangeShortName,industry,website,description,ceo,sector,country,fullTimeEmployees,phone,address,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund,date,label,adjDividend,dividend,recordDate,paymentDate,declarationDate
25,ASML,882.620,1.130000,1052273,348272468867,6.12000,564.0-883.42,14.870,ASML Holding N.V.,USD,0000937966,USN070592100,N07059210,NASDAQ Global Select,NASDAQ,Semiconductor Equipment & Materials,https://www.asml.com,"ASML Holding N.V. develops, produces, markets,...",Mr. Peter T. F. M. Wennink,Technology,NL,40310,31 480 235 0934,De Run 6501,Veldhoven,None,5504 DR,367.42869,502.571315,https://financialmodelingprep.com/image-stock/...,1995-03-15,False,False,True,True,False,2024-04-26,"April 26, 24",1.61438,1.61438,2024-04-29,2024-05-07,2024-01-26
26,ASML,882.620,1.130000,1052273,348272468867,6.12000,564.0-883.42,14.870,ASML Holding N.V.,USD,0000937966,USN070592100,N07059210,NASDAQ Global Select,NASDAQ,Semiconductor Equipment & Materials,https://www.asml.com,"ASML Holding N.V. develops, produces, markets,...",Mr. Peter T. F. M. Wennink,Technology,NL,40310,31 480 235 0934,De Run 6501,Veldhoven,None,5504 DR,367.42869,502.571315,https://financialmodelingprep.com/image-stock/...,1995-03-15,False,False,True,True,False,2024-02-05,"February 05, 24",1.34250,1.34250,2024-02-06,2024-02-14,2024-01-24
50,TPDNF,45.290,0.207662,0,4009025510,21.67409,45.29-45.29,0.000,Topdanmark A/S,USD,None,DK0060477503,None,Other OTC,PNK,Insurance—Diversified,https://www.topdanmark.com,"Topdanmark A/S, together with its subsidiaries...",Mr. Peter Hermann,Financial Services,DK,2100,45 44 68 33 11,Borupvang 4,Ballerup,None,2750,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2020-09-25,True,False,False,False,False,2024-04-24,"April 24, 24",1.67409,1.67409,2024-04-25,2024-04-26,None
52,CHDRY,201.183,1.065000,154,145182556355,3.41000,166.38-243.35,1.417,Christian Dior SE,USD,None,US1707151064,None,Other OTC,PNK,Luxury Goods,https://www.dior-finance.com,"Christian Dior SE, through its subsidiaries, e...",Mr. Antoine Arnault,Consumer Cyclical,FR,146479,33 1 44 13 22 22,"30, avenue Montaigne",Paris,None,75008,-491.92444,667.249443,https://financialmodelingprep.com/image-stock/...,2013-12-10,False,False,True,False,False,2024-04-23,"April 23, 24",2.03700,2.03700,2024-04-24,None,None
56,SEMHF,56.460,0.688000,309,62997396916,1.02000,47.99-58.57,1.380,Siemens Healthineers AG,USD,None,DE000SHL1006,None,Other OTC,PNK,Medical Devices,https://www.siemens-healthineers.com,"Siemens Healthineers AG, through its subsidiar...",Dr. Bernhard Montag,Healthcare,DE,71000,49 9131 84 3385,Henkestrasse 127,Erlangen,None,91052,46.98514,8.394858,https://financialmodelingprep.com/image-stock/...,2023-08-04,True,False,True,False,False,2024-04-19,"April 19, 24",1.03031,1.03031,2024-04-22,2024-04-23,None
67,ARBKL,10.000,0.000000,13162,123293600,6.56236,3.9-12.5,-0.050,Argo Blockchain plc 8.75% Senior Notes due 2026,USD,0001841675,US0401262037,None,NASDAQ Global Market,NASDAQ,None,None,None,,,US,None,None,None,None,None,None,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2021-11-18,True,False,True,False,False,2024-04-12,"April 12, 24",2.18750,2.18750,2024-04-15,2024-04-30,2023-12-19
69,WELPM,105.000,0.000000,20,0,7.50000,95.0-122.89,6.900,Wisconsin Electric Power Company PFD 6%,USD,0000107815,US9766562071,None,Other OTC,PNK,None,None,None,,,US,None,None,None,None,None,None,NaN,0.000000,https://financialmodelingprep.com/image-stock/...,2012-09-25,True,False,False,False,False,2024-04-11,"April 11, 24",1.50000,1.50000,2024-04-12,2024-04-30,None
136,CUYRF,75.320,1.108000,0,1297956645,181.21860,49.39-75.32,25.930,"C.Uyemura & Co.,Ltd.",USD,None,JP3155350006,None,Other OTC,PNK,Specialty Chemicals,https://www.uyemura.co.jp,"C.Uyemura & Co.,Ltd. engages in the manufactur...",Mr. Hiroya Uemura,Basic Materials,JP,1576,81 

It looks like after filtring, we come up with just a few rows - eight in fact. I honestly expected more and this is probably due to the fact that I ran up against the limit of the free api subscription.  
Okay, now that we have gotten the dividends registeredin the next six months, I have filtered for stocks in 'USD' and greater than $3.00. It seems that  there  could be  some other filters that  would be of interest, which come in the stock info payload like the following: `isEtf`, `isActivelyTrading`,	`isAdr`, `isFund`. This could possibly be worth  looking into with a paid subscription to the api.

We  have enough to go the last step, which is to plot one of the stocks along with the historical dividend trends. Let's see how to do this.
So, I'm doing this as a proof-of-concept, so I hope  it's not overly important,  which stock to choose. I'll pick `SEMHF`, because I know this stock - a few friends of mine work for this company ;).

In [55]:
semhf_historical_div=get_historical_dividends_for_symbol("SEMHF")

the url we are using: https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/SEMHF?apikey=4b0df7c34390a200aa6e767012b5950c
{
  "Error Message": "Limit Reach . Please upgrade your plan or visit our documentation for more details at https://site.financialmodelingprep.com/"
}


In [56]:
semhf_historical_price=get_historical_stock_price_for_symbol("SEMHF") 

the url we are using: https://financialmodelingprep.com/api/v3/historical-price-full/SEMHF?apikey=4b0df7c34390a200aa6e767012b5950c
{
  "Error Message": "Limit Reach . Please upgrade your plan or visit our documentation for more details at https://site.financialmodelingprep.com/"
}


Alrighty, it looks like the limit has been reached for the free api subscription. After looking at  the free subscription tier, it states that I have a limitof  250  calls a day. I blew the number of  calls on when  I got the stock  information   for all of the  stocks with upcoming dividends in  the next six months. For about  $30 a month, billed on a  monthly  basis, I can get 300 api calls a minute,which would fine for what I am  currently doing.  I would have  to  buy  about  30  stocks  of  Siemens  to achieve this at the current  price of $54.46 roughly equating to an investment of about $1,600, considering that the stock stays stable for that  period of time.  I won't be able to  look  at  the historical stock prices for this stock around the date of the dividend until 24 hrs have gone buy and I get another 200 free api calls  ;).   